In [1]:
import requests
import networkx
from time import sleep

In [2]:
AIRTABLE_KEY = ''

In [3]:
headers = {'Authorization': 'Bearer ' + AIRTABLE_KEY}
url = 'https://api.airtable.com/v0/appHwGHSloSDg4ukP/human_skeleton'
bones_data = {'records':[]}

In [4]:
def get_bone_data():
    all_responses = []
    response = requests.get(url, headers=headers)
    all_responses.append(response.json())
    
    while 'offset' in response.json():
        sleep(0.5)
        params = {'offset':response.json()['offset']}
        response = requests.get(url, headers=headers, params=params)
        all_responses.append(response.json())
        
    for recset in all_responses:
        bones_data['records'] = bones_data['records'] + recset['records']
        
get_bone_data()
print(len(bones_data['records']))

206


In [5]:
url = 'https://api.airtable.com/v0/appHwGHSloSDg4ukP/body_regions'
response = requests.get(url, headers=headers)
region_data = response.json()
print(len(region_data['records']))

28


In [6]:
region_lookup = {}

for r in region_data['records']:
    rec = {
        'key': r['fields']['key'],
        'id_number': r['fields']['id_number'],
        'name': r['fields']['region_name'],
        'node_type': r['fields']['region_type'],
        'temp_adj_regions': r['fields']['adjacent_regions'],
        'adjacent_regions': []
    }
    if 'side' in r['fields'].keys():
        rec['side'] = r['fields']['side']
        
    region_lookup[r['id']] = rec
     
for k,v in region_lookup.items():
    for trid in v['temp_adj_regions']:
        region_lookup[k]['adjacent_regions'].append(region_lookup[trid]['id_number'])
    del v['temp_adj_regions']
    
region_node_table = {}

for k, v in region_lookup.items():
    region_node_table[v['id_number']] = v

In [7]:
bone_lookup = {}

for r in bones_data['records']:
    rec = {
        'id_number': r['fields']['id_number'],
        'bone_name': r['fields']['bone_name'],
        'readable_name': r['fields']['readable_name'],
        'key': r['fields']['key'],
        'regions_temp': r['fields']['body_regions'],
        'body_regions': [],
        'temp_proximal': [],
        'temp_distal': [],
        'temp_adjacent': [],
        'proximal_bones': [],
        'distal_bones': [],
        'adjacent_bones': []
    }
    
    for br in rec['regions_temp']:
        rec['body_regions'].append(region_lookup[br]['id_number'])
        
    del rec['regions_temp']
    
    for k in ['side', 'alternate_name']:
        if k in r['fields'].keys():
            rec[k] = r['fields'][k]
    
    if 'proximal_bones' in r['fields'].keys():
        for b in r['fields']['proximal_bones']:
            rec['temp_proximal'].append(b)
    
    if 'distal_bones' in r['fields'].keys():
        for b in r['fields']['distal_bones']:
            rec['temp_distal'].append(b)
            
    if 'adjacent_bones' in r['fields'].keys():
        for b in r['fields']['adjacent_bones']:
            rec['temp_adjacent'].append(b)
    
    bone_lookup[r['id']] = rec
    
for k, v in bone_lookup.items():
    for b in v['temp_proximal']:
        v['proximal_bones'].append(bone_lookup[b]['id_number'])
    for b in v['temp_distal']:
        v['distal_bones'].append(bone_lookup[b]['id_number'])
    for b in v['temp_adjacent']:
        v['adjacent_bones'].append(bone_lookup[b]['id_number'])
        
for k, v in bone_lookup.items():
    del v['temp_proximal']
    del v['temp_distal']
    del v['temp_adjacent']

In [8]:
bone_nodes = {}
for k,v in bone_lookup.items():
    bone_nodes[v['id_number']] = v

### Create a graph

In [9]:
skeleton = networkx.DiGraph()

In [10]:
# Add regions
for k,v in region_node_table.items():
    tup = (k,{
        "name": v['name'],
        "node_type": v['node_type']
    })
    if "side" in v.keys():
        tup[1]["side"] = v["side"]
    skeleton.add_node(tup[0], tup[1])

In [11]:
# Add edges between regions
for k, v in region_node_table.items():
    for n in v['adjacent_regions']:
        skeleton.add_edge(k, n, edge_type="adjacent")

In [12]:
for k, v in bone_nodes.items():
    tup = (k, {
        "name": v['bone_name'],
        "node_type": "bone"
    })
    
    for key in ["readable_name", "alternate_name", "position", "side"]:
        if key in v.keys():
            tup[1][key] = v[key]
            
    skeleton.add_node(tup[0], tup[1])

In [13]:
for k, v in bone_nodes.items():
    # other bones
    if 'proximal_bones' in v.keys():
        for n in v['proximal_bones']:
            skeleton.add_edge(k, n, edge_type="proximal")
    if 'distal_bones' in v.keys():
        for n in v['distal_bones']:
            skeleton.add_edge(k, n, edge_type="distal")
    if 'adjacent_bones' in v.keys():
        for n in v['adjacent_bones']:
            skeleton.add_edge(k, n, edge_type="adjacent")
            
    # body regions
    for n in v['body_regions']:
        skeleton.add_edge(k, n, edge_type="region")
        skeleton.add_edge(n, k, edge_type="bone")

## Export data

From the docs: https://networkx.github.io/documentation/latest/reference/readwrite/json_graph.html

In [14]:
from networkx.readwrite import json_graph
import json

In [15]:
d = json_graph.node_link_data(skeleton)

In [17]:
with open("skeleton_directed_node_link_graph.json", "w") as f:
    f.write(json.dumps(d))

In [18]:
# G = skeleton.to_undirected()

In [19]:
# gout = json_graph.node_link_data(G)

In [20]:
# with open("skeleton_undirected_node_link_graph.json", "w") as f:
#     f.write(json.dumps(gout))